In [1]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

C:\Users\saltuscorp\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Constructing model

In [2]:
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32, name='X')
Y = tf.placeholder(tf.float32, name='Y')

with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1.0, 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1.0, 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1.0, 1.), name='W3')
    model = tf.matmul(L2, W3)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step = global_step)
    
    ###손실 값을 추적하기 위해 수집할 값을 지정하는 코드
    tf.summary.scalar('cost', cost)
    #######################

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



# Training model

In [4]:
with tf.Session() as sess:
    saver = tf.train.Saver(tf.global_variables())
    ckpt = tf.train.get_checkpoint_state('./model')
    if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(tf.global_variables_initializer())
    
    ######
    merged = tf.summary.merge_all()    
    writer = tf.summary.FileWriter('./logs', sess.graph)
    ######
    
    for step in range(100):
        sess.run(train_op, feed_dict={X: x_data, Y: y_data})
        
        print('Step : %d' % sess.run(global_step),
                'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))
        
        summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary, global_step=sess.run(global_step))
        
    saver.save(sess, './model/dnn.ckpt', global_step=global_step)
    
    prediction = tf.argmax(model, 1)
    target = tf.argmax(Y, 1)
    print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
    print('실제값:', sess.run(target, feed_dict={Y: y_data}))
    
    is_correct = tf.equal(prediction, target)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    
    print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

Step : 1 Cost: 1.256
Step : 2 Cost: 1.157
Step : 3 Cost: 1.073
Step : 4 Cost: 1.000
Step : 5 Cost: 0.939
Step : 6 Cost: 0.888
Step : 7 Cost: 0.844
Step : 8 Cost: 0.806
Step : 9 Cost: 0.773
Step : 10 Cost: 0.744
Step : 11 Cost: 0.720
Step : 12 Cost: 0.698
Step : 13 Cost: 0.681
Step : 14 Cost: 0.665
Step : 15 Cost: 0.651
Step : 16 Cost: 0.639
Step : 17 Cost: 0.628
Step : 18 Cost: 0.619
Step : 19 Cost: 0.610
Step : 20 Cost: 0.603
Step : 21 Cost: 0.596
Step : 22 Cost: 0.590
Step : 23 Cost: 0.586
Step : 24 Cost: 0.581
Step : 25 Cost: 0.577
Step : 26 Cost: 0.574
Step : 27 Cost: 0.571
Step : 28 Cost: 0.569
Step : 29 Cost: 0.567
Step : 30 Cost: 0.565
Step : 31 Cost: 0.563
Step : 32 Cost: 0.562
Step : 33 Cost: 0.561
Step : 34 Cost: 0.559
Step : 35 Cost: 0.559
Step : 36 Cost: 0.558
Step : 37 Cost: 0.557
Step : 38 Cost: 0.556
Step : 39 Cost: 0.556
Step : 40 Cost: 0.555
Step : 41 Cost: 0.555
Step : 42 Cost: 0.554
Step : 43 Cost: 0.554
Step : 44 Cost: 0.554
Step : 45 Cost: 0.554
Step : 46 Cost: 0.5

C:\Users\saltuscorp\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Step: 1,  Cost: 1.426


InvalidArgumentError: You must feed a value for placeholder tensor 'X' with dtype float
	 [[Node: X = Placeholder[dtype=DT_FLOAT, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'X', defined at:
  File "C:\Users\saltuscorp\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\saltuscorp\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\saltuscorp\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\saltuscorp\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\saltuscorp\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Users\saltuscorp\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
  File "C:\User